In [1]:
import openai

with open("../API_KEY", "r") as f:
    key = f.read()

openai.api_key = key

# General Plan:
- Make a function that converts a python function into the json representation required by openai
- Then allow for making a chain where GPT mostly creates functions for itself:
    - Get gpt to make a function
    - Get gpt to save that function in a folder along with the json representation of the function
    - Automatically include all tools in the folder (i.e. using the json representations, but then being able to call the functions)
    

## First, let's test out the function calling

Make a very basic function first

In [ ]:
def get_weather_report(day_of_week: int, weather_type: str, temperature: float = 10.0) -> str:
    """
    Converts information about weather into a string representation.

    Args:
        day_of_week (int): The day of the week from 0 to 6.
        weather_type (str): The type of weather, can be "sunny", "rainy", or "windy".
        temperature (float, optional): Temperature in Celsius. Defaults to 10.0.

    Returns:
        str: A string representation of the weather report.
    """
    return f'For the {day_of_week}th day of the week, the weather is predicted to be {weather_type} with a max temperature of {temperature}'

Manually create the JSON representation for that function

In [ ]:

weather_report_function = {
    "name": "get_weather_report",
    "description": "Converts information about weather into a string representation",
    "parameters": {
        "type": "object",
        "properties": {
            "day_of_week": {
                "type": "number",
                "description": "The day of the week from 0 to 6",
            },
            "weather_type": {"type": "string", "enum": ["sunny", "rainy", "windy"]},
            "temperature": {"type": "number", "description": "Temperature in Celsius. Defaults to 10.0."},
        },
        "required": ["day_of_week", "weather_type"],
    },
}



Check that openai uses the function call

In [ ]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[{"role": "user", "content": "It's currently a sunny 36.4C this Tuesday, can you give me a weather report?"}],
    functions=[
        weather_report_function,
    ],
    function_call="auto",
)
response

Then use the result of the function call to generate the next response in the chain

In [ ]:
message = response['choices'][0]['message']
if message.get("function_call"):
    print(f'Chat model responded with a function call:\n{message.get("function_call")}')
    print(f'\nNow determining next output including function return')
    function_name = message["function_call"]["name"]

    # Step 3, call the function
    # Note: the JSON response from the model may not be valid JSON
    function_response = get_weather_report(
        **json.loads(message['function_call'].get('arguments')),
    )
    print(f'Function output:\n{function_response}')

    # Step 4, send model the info on the function call and function response
    second_response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[
            {"role": "user", "content": "It's currently 36.4C this Tuesday, can you give me a weather report?"},
            message,
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            },
        ],
    )
    print(f"Next response from Chat model:\n{second_response['choices'][0]['message']}")
elif message.get('content'):
    print(f'Chat model responded directly with:\n{message.get("content")}')
    

## Basic test worked, now let's start automating

Below is a function that should convert a given function into the json representation of that function

In [ ]:
import inspect
import json
import numpy
from typing import Callable

# def get_json_representation(func: Callable) -> dict:
#     """
#     Returns a JSON representation of the given function.

#     Args:
#         func (Callable): The function to represent.

#     Returns:
#         str: The JSON representation of the function.
#     """
#     signature = inspect.signature(func)
#     docstring = inspect.getdoc(func)

#     # Parse the docstring
#     docstring_lines = docstring.split('\n')
#     description = docstring_lines[0]
#     parameter_descriptions = {}
#     args_index = -1
#     for line in docstring_lines[1:]:
#         if line.strip().startswith("Args:"):
#             args_index = docstring_lines.index(line)
#             break
#     if args_index != -1:
#         for line in docstring_lines[args_index+1:]:
#             if line.strip().startswith(tuple([f"{param}" for param in signature.parameters.keys()])):
#                 param, desc = line.strip().split(":", 1)
#                 param = param.split(" (")[0]  # Remove type hinting if present
#                 parameter_descriptions[param] = desc.strip()

#     # Map Python types to JSON types
#     type_mapping = {str: 'string', bool: 'boolean'}

#     # Build the JSON representation
#     json_representation = {
#         'name': func.__name__,
#         'description': description,
#         'parameters': {
#             'type': 'object',
#             'properties': {
#                 name: {
#                     'type': type_mapping.get(param.annotation, 'any') if not numpy.issubdtype(param.annotation, numpy.number) else 'number',
#                     'description': parameter_descriptions.get(name, '')
#                 }
#                 for name, param in signature.parameters.items()
#             },
#             'required': [name for name, param in signature.parameters.items() if param.default is param.empty]
#         }
#     }

#     return json_representation

def get_json_representation(func: Callable) -> dict:
    """
    Uses the openai.ChatCompletion.create endpoint to return the necessary imports for a function supplied as a string.

    Args:
        function_str (str): The function for which to get the necessary imports, supplied as a string.

    Returns:
        str: A string of the necessary imports for the function.
    """
    parameters = {
        'model': 'gpt-3.5-turbo-0613',
        'temperature': 0.0,
        'messages': [
            {"role": "system", "content": '''Your job is to convert a python function into a json representation with a specific form.
For example, given this function:
```

def get_weather_report(day_of_week: int, weather_type: str, temperature: float = 10.0) -> str:
    """
    Converts information about weather into a string representation.

    Args:
        day_of_week (int): The day of the week from 0 to 6.
        weather_type (str): The type of weather, can be "sunny", "rainy", or "windy".
        temperature (float, optional): Temperature in Celsius. Defaults to 10.0.

    Returns:
        str: A string representation of the weather report.
    """
    return f'For the {day_of_week}th day of the week, the weather is predicted to be {weather_type} with a max temperature of {temperature}'
```

You should return:
```
{
    "name": "get_weather_report",
    "description": "Converts information about weather into a string representation",
    "parameters": {
        "type": "object",
        "properties": {
            "day_of_week": {
                "type": "number",
                "description": "The day of the week from 0 to 6",
            },
            "weather_type": {"type": "string", "enum": ["sunny", "rainy", "windy"]},
            "temperature": {"type": "number", "description": "Temperature in Celsius. Defaults to 10.0."},
        },
        "required": ["day_of_week", "weather_type"],
    },
}
```
Return the JSON ONLY.'''},
            {"role": "user", "content": inspect.getsource(func)}
        ]
    }

    response = openai.ChatCompletion.create(**parameters)

    # Extract the assistant's response
    assistant_response = response['choices'][0]['message']['content']

    return json.loads(assistant_response)

In [ ]:
json_rep = get_json_representation(get_weather_report)
print(json.dumps(json_rep, indent=4))

---
That seems to work reasonably well, so now let's make sure we get the same responses from the LLM using this

In [ ]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[{"role": "user", "content": "It's currently a sunny 36.4C this Tuesday, can you give me a weather report?"}],
    functions=[
        get_json_representation(get_weather_report),
    ],
    function_call="auto",
)
response

Then use the result of the function call to generate the next response in the chain

In [ ]:
message = response['choices'][0]['message']
if message.get("function_call"):
    print(f'Chat model responded with a function call:\n{message.get("function_call")}')
    print(f'\nNow determining next output including function return')
    function_name = message["function_call"]["name"]

    # Step 3, call the function
    # Note: the JSON response from the model may not be valid JSON
    function_response = get_weather_report(
        **json.loads(message['function_call'].get('arguments')),
    )
    print(f'Function output:\n{function_response}')

    # Step 4, send model the info on the function call and function response
    second_response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[
            {"role": "user", "content": "It's currently 36.4C this Tuesday, can you give me a weather report?"},
            message,
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            },
        ],
    )
    print(f"Next response from Chat model:\n{second_response['choices'][0]['message']}")
elif message.get('content'):
    print(f'Chat model responded directly with:\n{message.get("content")}')
    

In [ ]:
import os
functions_folder = 'functions'
os.makedirs(functions_folder, exist_ok=True)


## Make a function that saves the func/description to file

In [ ]:
import json
from typing import Callable

def get_imports_for_function(function_str: str) -> str:
    """
    Uses the openai.ChatCompletion.create endpoint to return the necessary imports for a function supplied as a string.

    Args:
        function_str (str): The function for which to get the necessary imports, supplied as a string.

    Returns:
        str: A string of the necessary imports for the function.
    """
    parameters = {
        'model': 'gpt-3.5-turbo-0613',
        'messages': [
            {"role": "system", "content": """Your job is to return ONLY the python imports that would be required to run the function given by the user. For example:
            -----
            The user has given you this function:
            ```
            def some_function(a, b, **kwargs):
                c = np.sin(a+b)
                result = {'input_a': a, 'input_b': b, 'res': c}
                return json.dumps(result)
            ```
            You should return:
            ```
            import numpy as np
            import json
            ```
            -----
            
            Do not include ANY other text other than the imports required. If no imports are required return "No imports required".
            """},
            {"role": "user", "content": f"I have a function: {function_str}. What are the necessary imports for this function?"}
        ]
    }

    response = openai.ChatCompletion.create(**parameters)

    # Extract the assistant's response
    assistant_response = response['choices'][0]['message']['content']

    return assistant_response

In [ ]:
import numpy as np
import json 

def test_func(d: dict, a: int, b: float) -> str:
    """
    Calculates the sine of the product of a and b and adds it to the dictionary d with the key 'new_val'.

    Args:
      d: A dictionary to which the new value will be added.
      a: An integer value.
      b: A float value.

    Returns:
      A JSON string representation of the updated dictionary d.
    """
    d['new_val'] = np.sin(a*b)
    return json.dumps(d)

test_func({}, 5, 10)

In [ ]:
get_imports_for_function(inspect.getsource(test_func))

In [ ]:
def test_func2(a: float, b: float) -> float:
    """
    Multiplies two float values and returns the result.

    Args:
      a: A float value.
      b: A float value.

    Returns:
      The product of a and b.
    """
    return a*b

In [ ]:
get_imports_for_function(inspect.getsource(test_func2))

In [ ]:
def save_func_and_json(func: Callable, func_folder: str) -> None:
    """
    Saves the given function and its JSON representation to a subfolder of func_folder.

    Args:
        func (Callable): The function to save.
        func_folder (str): The path to the folder where the function and JSON representation will be saved.
    """
    # Get the imports required for the function
    imports_string = get_imports_for_function(inspect.getsource(func))
    if 'no imports required' in imports_string.lower():
        imports_string = ''
    else:
        imports_string += '\n\n\n'
        
    file_contents = imports_string+inspect.getsource(func)
    
    # Save function to .py file
    with open(f"{func_folder}/{func.__name__}.py", "w") as f:
        f.write(file_contents)

    # Save JSON representation to .json file
    with open(f"{func_folder}/{func.__name__}.json", "w") as f:
        json.dump(get_json_representation(func), f)

In [ ]:
save_func_and_json(test_func, func_folder=functions_folder)

In [ ]:
save_func_and_json(test_func2, func_folder=functions_folder)

# TODOs:
- Load all descriptions from folder to pass into completion
- Run the functions in the files when completion requests function call
- Tidy up into something nice
- Integrate this with asking GPT to make a new function (and automatically save etc)

In [ ]:
import os
import json

def load_json_files(directory: str) -> dict:
    """
    Loads the contents of all the .json files in a directory.

    Args:
        directory (str): The directory to load .json files from.

    Returns:
        dict: A dictionary where the key is the name of the json file (excluding the .json extension) and the value is the content of the .json file.
    """
    json_data = {}

    # Iterate over all files in the directory
    for filename in os.listdir(directory):
        # Check if the file is a .json file
        if filename.endswith(".json"):
            # Remove the .json extension from the filename
            name = filename[:-5]
            # Open the .json file and load its contents
            with open(os.path.join(directory, filename), 'r') as f:
                data = json.load(f)
            # Add the data to the dictionary
            json_data[name] = data

    return json_data



In [ ]:
function_descriptions = load_json_files('functions')
function_descriptions

In [ ]:
import importlib.util

def load_function_from_file(directory: str, filename: str):
    """
    Loads a function from a .py file.

    Args:
        directory (str): The directory where the .py file is located.
        filename (str): The name of the .py file (excluding the .py extension).

    Returns:
        function: The function contained in the .py file.
    """
    # Create the path to the .py file
    file_path = os.path.join(directory, filename + ".py")

    # Load the spec of the module
    spec = importlib.util.spec_from_file_location(filename, file_path)

    # Create a module from the spec
    module = importlib.util.module_from_spec(spec)

    # Execute the module to get the function
    spec.loader.exec_module(module)

    # Get the function from the module
    function = getattr(module, filename)

    return function


In [ ]:
funcs = {}
for func_name in function_descriptions:
    funcs[func_name] = load_function_from_file('functions', func_name)
funcs

# Now let's provide a way for GPT to make it's own function, save it, and use it in its next response!

- Make a write python code function (the function should just take a description of what the code needs to do so that a separate prompt can be used to actually generate the code)

- Note: Maybe can improve the code generator by making it fill in a fake function that takes args for 'signature', 'docstring', 'code body' or something like that?

In [ ]:
def make_new_function(function_name: str, arg_descriptions: str, description: str):
    """
    Use this to make a new function with given `function_name` that will then be accessible to use for future messages. The new function will be made to carry out the task described in the `description` given the arguments described by `arg_descriptions`
    
    Args:
        function_name: Name to give the new function (should follow python naming conventions)
        arg_descriptions: A description of any arguments that the function should take (including type, and default value if appropriate)
        description: A description of what the function should do (including the what it should output)
    """
    parameters = {
        'model': 'gpt-3.5-turbo-0613',
        'temperature': 0.0,
        'messages': [
            {"role": "system", "content": """You are an expert python coder that will be tasked with generating the code to go in a .py file for a single function given some specific information from the user.
You will be provided:
    - function_name: The name to give the new function
    - arg_descriptions: Descriptions of all the arguments the function should take (if their types are missing, try to infer them)
    - description: What the function should do with the given arguments

When generating the new function you should follow these rules:
    - Include ONLY the text that will be in the python file (e.g. starting with `import ...` unless no imports are necessary in which case, starting with `def ...`)
    - Do NOT include any plain text explanation at the end of the written code
    - Use the latest python programming techniques and best practices
    - Use the latest/best python libraries when appropriate (e.g. if plotting, use `plotly` instead of `matplotlib` because plotly is better library even though matplotlib is better known)
    - Always include a google style docstring
    - Include type hints for the inputs and output
    - Include all necessary imports
    - Do NOT use markdown backticks to surround your output, return ONLY the contents of the .py file
"""},
            {"role": "user", "content": f"function_name: {function_name}\narg_descriptions: {arg_descriptions}\ndescription: {description}"}
        ]
    }

    response = openai.ChatCompletion.create(**parameters)

    # Extract the assistant's response
    function_code = response['choices'][0]['message']['content']
    return function_code
    


In [ ]:
make_new_function_json = get_json_representation(make_new_function)
make_new_function_json

In [ ]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[{"role": "user", "content": "I'd like you to multiply a list of numbers together and return the total value, the list is [3,2,6,3,6,5,4,3,6]."}],
    functions=[
        make_new_function_json,
    ],
    # function_call="auto",
    function_call={"name": "make_new_function"},  # Force using the function
)
response

In [ ]:
from typing import Callable
def sanitize_python_code(code: str) -> str:
    """
    Sanitizes a string containing Python code. If the code is surrounded by markdown triple backticks, they are removed.
    Also, language identifiers immediately following the opening backticks (like 'python' or 'py') are removed.

    Args:
        code (str): The string containing Python code.

    Returns:
        str: The sanitized Python code.
    """
    # Check if the string starts and ends with triple backticks
    if code.startswith("```") and code.endswith("```"):
        # Remove the triple backticks from the start and end of the string
        code = code[3:-3]
        
    # Further check if the string starts with "python" or "py", which is common in markdown code blocks
    if code.lstrip().startswith(("python", "py")):
        # Find the first newline character and remove everything before it
        code = code[code.find('\n')+1:]

    return code

def write_to_py_file(folder: str, file_name: str, file_contents: str):
    # Save function to .py file
    file_contents = sanitize_python_code(file_contents)
    with open(f"{folder}/{file_name}.py", "w") as f:
        f.write(file_contents)

def write_description_to_json_file(folder: str, func: Callable):
    # Save JSON representation to .json file
    with open(f"{folder}/{func.__name__}.json", "w") as f:
        json.dump(get_json_representation(func), f)
        
def write_generated_func_to_file(folder: str, func_name: str, generated_file_contents: str):
    write_to_py_file(folder, func_name, generated_file_contents)
    func = load_function_from_file(folder, func_name)
    write_description_to_json_file(folder, func)

In [ ]:
print(json.dumps(func_descriptions['multiply_list'], indent=4))
print()
print(json.dumps(func_descriptions['test_func'], indent=4))

In [ ]:
message = response['choices'][0]['message']

print(f'Chat model responded with a function call:\n{message.get("function_call")}')
print(f'\nNow generating new function')
called_function = message["function_call"]["name"]

# Step 3, call the function
# Note: the JSON response from the model may not be valid JSON
new_file_contents = make_new_function(
    **json.loads(message['function_call'].get('arguments')),
)
print(f'Function output:\n{new_file_contents}')

new_func_name = json.loads(message['function_call']['arguments']).get('function_name')
write_generated_func_to_file(functions_folder, new_func_name, new_file_contents)
print(f'Funcion written to file')

func_descriptions = load_json_files(functions_folder)
print(f'Function description written to file')
   

In [ ]:
# func_descriptions['multiply_list']['parameters']['properties']['numbers']['type'] = 'object'
# del func_descriptions['multiply_list']['parameters']['properties']['numbers']['items']
# del func_descriptions['multiply_list']
func_descriptions

In [ ]:
 
# Step 4, send model the info on the function call and function response
second_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613", 
    messages=[{"role": "user", "content": "I'd like you to multiply a list of numbers together and return the total value, the list is [3,2,6,3,6,5,4,3,6]."},
        # message,
        # {
        #     "role": "function",
        #     "name": called_function,
        #     "content": function_response,
        # },
    ],
    functions=[
        make_new_function_json,
        *func_descriptions.values(),
    ]
)
print(f"Next response from Chat model:\n{second_response['choices'][0]['message']}")
    

**NOTE:** We don't want to follow the standard format here of telling GPT what the function returned, we just want to pretend that we are again answering the original question, only now we have the new function available

In [ ]:
message = second_response.choices[0]['message']
print(f'Chat model responded with a function call:\n{message.get("function_call")}')
print(f'\nNow determining next output including function return')
function_name = message["function_call"]["name"]

# Step 3, call the function
# Note: the JSON response from the model may not be valid JSON
func = load_function_from_file(functions_folder, function_name)
print(f'Using func: {func}')
function_response = func(
    **json.loads(message['function_call'].get('arguments')),
)
print(f'Function output:\n{function_response}')

# Step 4, send model the info on the function call and function response
second_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role": "user", "content": "I'd like you to multiply a list of numbers together and return the total value, the list is [3,2,6,3,6,5,4,3,6]."},
        message,
        {
            "role": "function",
            "name": function_name,
            "content": str(function_response),
        },
    ],
)
print(f"Next response from Chat model:\n{second_response['choices'][0]['message']}")